In [2]:
import csv
import time
import os
import subprocess
import psutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization



class CustomCallback(keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()
        self.training_time = None
        self.average_memory_utilization = None
        self.average_gpu_utilization = None
        
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        self.memory_utilizations = []
        self.gpu_utilizations = []
        
    def on_train_end(self, logs={}):
        self.training_time = time.time() - self.start_time
        self.average_memory_utilization = np.mean(self.memory_utilizations)
        self.average_gpu_utilization = np.mean(self.gpu_utilizations)
        print('\nTotal training time: {} seconds'.format(self.training_time))
        print('\nAverage memory utilization during training so far: {}'.format(self.average_memory_utilization))
        print('\nAverage GPU utilization during training so far: {}'.format(self.average_gpu_utilization))

    def on_epoch_end(self, epoch, logs={}):
        memory_utilization = psutil.virtual_memory().used / pow(10, 9) # ex. 8.6Gi
        gpu_utilization_process = subprocess.Popen(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader'], stdout=subprocess.PIPE)
        gpu_utilization = int(gpu_utilization_process.stdout.readline().strip().decode('utf-8').split()[0])
        self.memory_utilizations.append(memory_utilization)
        self.gpu_utilizations.append(gpu_utilization)

        
def Classify(batch_size, epochs):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    callback = CustomCallback()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epoch, validation_data=(x_test, y_test), callbacks=[callback])
    
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    
    with open('cifar10_deep_classification.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([batch_size, epochs, callback.average_memory_utilization, callback.average_gpu_utilization, callback.training_time, test_loss, test_acc])

        
for batch in range(100, 801, 100):
    for epoch in range(10, 41, 10):
        Classify(batch, epoch)


Epoch 1/10


2023-02-11 02:02:18.090992: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2023-02-11 02:02:18.530512: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-11 02:02:18.531069: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-11 02:02:18.531130: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-11 02:02:18.531575: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-11 02:02:18.531632: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


500/500 [==============================] - 8s 13ms/step - loss: 1.6926 - accuracy: 0.4273 - val_loss: 1.4896 - val_accuracy: 0.4646
Epoch 2/10
500/500 [==============================] - 6s 12ms/step - loss: 1.1115 - accuracy: 0.6057 - val_loss: 0.9869 - val_accuracy: 0.6476
Epoch 3/10
500/500 [==============================] - 6s 12ms/step - loss: 0.9125 - accuracy: 0.6799 - val_loss: 0.7970 - val_accuracy: 0.7260
Epoch 4/10
500/500 [==============================] - 6s 12ms/step - loss: 0.8006 - accuracy: 0.7200 - val_loss: 0.7076 - val_accuracy: 0.7509
Epoch 5/10
500/500 [==============================] - 6s 12ms/step - loss: 0.7243 - accuracy: 0.7501 - val_loss: 0.7286 - val_accuracy: 0.7463
Epoch 6/10
500/500 [==============================] - 6s 12ms/step - loss: 0.6592 - accuracy: 0.7702 - val_loss: 0.6976 - val_accuracy: 0.7618
Epoch 7/10
500/500 [==============================] - 6s 12ms/step - loss: 0.6202 - accuracy: 0.7861 - val_loss: 0.5638 - val_accuracy: 0.8078
Epoch 8/10

2023-02-11 02:37:50.951006: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 615.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-02-11 02:37:50.951049: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 615.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-02-11 02:37:50.994130: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 615.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-02-11 02:37:50.994162: W tensorflow/core/common_runtime/bfc_allocator.cc:275]

  3/100 [..............................] - ETA: 4s - loss: 3.4653 - accuracy: 0.1253  

...

100/100 [==============================] - 6s 50ms/step - loss: 1.9871 - accuracy: 0.3606 - val_loss: 3.2116 - val_accuracy: 0.1000
Epoch 2/10
100/100 [==============================] - 5s 45ms/step - loss: 1.3438 - accuracy: 0.5201 - val_loss: 3.0831 - val_accuracy: 0.1228
Epoch 3/10
100/100 [==============================] - 4s 45ms/step - loss: 1.1181 - accuracy: 0.6024 - val_loss: 2.9311 - val_accuracy: 0.2272
Epoch 4/10
100/100 [==============================] - 5s 45ms/step - loss: 0.9867 - accuracy: 0.6489 - val_loss: 2.3999 - val_accuracy: 0.3106
Epoch 5/10
100/100 [==============================] - 5s 45ms/step - loss: 0.8929 - accuracy: 0.6853 - val_loss: 1.4143 - val_accuracy: 0.5134
Epoch 6/10
100/100 [==============================] - 5s 45ms/step - loss: 0.8223 - accuracy: 0.7089 - val_loss: 0.8539 - val_accuracy: 0.7027
Epoch 7/10
100/100 [==============================] - 5s 45ms/step - loss: 0.7626 - accuracy: 0.7340 - val_loss: 0.7329 - val_accuracy: 0.7436
Epoch 8/10

2023-02-11 02:45:40.794619: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-02-11 02:45:40.794654: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


84/84 [==============================] - 6s 62ms/step - loss: 1.9886 - accuracy: 0.3566 - val_loss: 3.0312 - val_accuracy: 0.1030
Epoch 2/10
84/84 [==============================] - 5s 55ms/step - loss: 1.3994 - accuracy: 0.5041 - val_loss: 3.5397 - val_accuracy: 0.1000
Epoch 3/10
84/84 [==============================] - 5s 55ms/step - loss: 1.1586 - accuracy: 0.5862 - val_loss: 3.0813 - val_accuracy: 0.1697
Epoch 4/10
84/84 [==============================] - 5s 55ms/step - loss: 1.0208 - accuracy: 0.6388 - val_loss: 2.7506 - val_accuracy: 0.2699
Epoch 5/10
84/84 [==============================] - 5s 55ms/step - loss: 0.9167 - accuracy: 0.6739 - val_loss: 1.7506 - val_accuracy: 0.4215
Epoch 6/10
84/84 [==============================] - 5s 55ms/step - loss: 0.8480 - accuracy: 0.7011 - val_loss: 1.4366 - val_accuracy: 0.5261
Epoch 7/10
84/84 [==============================] - 5s 55ms/step - loss: 0.7874 - accuracy: 0.7251 - val_loss: 0.8964 - val_accuracy: 0.6856
Epoch 8/10
84/84 [======

2023-02-11 02:53:41.952955: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.51GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-02-11 02:53:41.952998: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.51GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


72/72 [==============================] - 6s 72ms/step - loss: 2.0277 - accuracy: 0.3563 - val_loss: 2.8119 - val_accuracy: 0.1000
Epoch 2/10
72/72 [==============================] - 5s 64ms/step - loss: 1.4048 - accuracy: 0.5069 - val_loss: 3.3501 - val_accuracy: 0.1000
Epoch 3/10
72/72 [==============================] - 5s 64ms/step - loss: 1.1618 - accuracy: 0.5905 - val_loss: 3.8162 - val_accuracy: 0.1319
Epoch 4/10
72/72 [==============================] - 5s 64ms/step - loss: 1.0149 - accuracy: 0.6404 - val_loss: 3.7929 - val_accuracy: 0.1397
Epoch 5/10
72/72 [==============================] - 5s 64ms/step - loss: 0.9192 - accuracy: 0.6755 - val_loss: 3.7601 - val_accuracy: 0.1859
Epoch 6/10
72/72 [==============================] - 5s 64ms/step - loss: 0.8537 - accuracy: 0.7014 - val_loss: 2.7585 - val_accuracy: 0.2232
Epoch 7/10
72/72 [==============================] - 5s 64ms/step - loss: 0.7992 - accuracy: 0.7205 - val_loss: 1.5764 - val_accuracy: 0.4763
Epoch 8/10
72/72 [======